# Trabajo Practico 2 - Grupo 04 - Backbenchers


Integrantes:

* Catalina Basso
* Cristobal Alvarez
* Lucas Ruiz
* Dalmiro Vilaplana


## Imports

In [ ]:
import re
import string
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV, StratifiedKFold, KFold
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, KFold, RandomizedSearchCV

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

## Funciones

In [ ]:
def limpiar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'http\S+|www\S+|https\S+', '', texto, flags=re.MULTILINE)
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    texto = re.sub(r'\d+', '', texto)
    stop_words = set(stopwords.words('spanish'))
    stemmer = SnowballStemmer('spanish')
    palabras = texto.split()
    palabras = [stemmer.stem(palabra) for palabra in palabras if palabra not in stop_words]
    texto = ' '.join(palabras)
    return texto

def clean_text(text):
    text = text.lower()
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub("\d+", "", text)
    text = text.strip()
    return text

stop_words = set(stopwords.words('spanish'))

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return " ".join(filtered_words)

def limpiar_texto_leman(texto):
    texto = re.sub(r'[^\w\s]', '', texto)
    texto = texto.lower()
    texto = re.sub(r'\d+', '', texto)
    stop_words = set(stopwords.words('spanish'))
    palabras = texto.split()
    palabras = [palabra for palabra in palabras if palabra not in stop_words]
    lemmatizer = WordNetLemmatizer()
    palabras = [lemmatizer.lemmatize(palabra) for palabra in palabras]
    return ' '.join(palabras)

stemmer = SnowballStemmer('spanish')

def stem_text(text):
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return " ".join(stemmed_words)

def crear_bag_of_words(textos):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(textos)
    return X, vectorizer

def crear_tfidf(textos, max_features=10000, ngram_range=(1, 2)):
    vectorizer = TfidfVectorizer(max_features=max_features, ngram_range=ngram_range)
    X = vectorizer.fit_transform(textos)
    return X, vectorizer

## Lecura CSV



Primero cargamos y vemos nuestro dataset

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tp2_train.csv')
train.head()

ID                                          review_es sentimiento
0   0  Uno de los otros críticos ha mencionado que de...    positivo
1   1  Una pequeña pequeña producción.La técnica de f...    positivo
2   2  Pensé que esta era una manera maravillosa de p...    positivo
3   3  Básicamente, hay una familia donde un niño peq...    negativo
4   4  El "amor en el tiempo" de Petter Mattei es una...    positivo

In [ ]:
train.shape

(50000, 3)

In [ ]:
test_kaggle = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tp2_test.csv')
test_kaggle.head()

ID                                          review_es
0  60000  La mayor virtud de esta película es su existen...
1  60001  No soy un experto cinéfilo, pero pocas veces m...
2  60002  Si no eres un incondicional del humor estilo T...
3  60003  No sé qué está pasando, si la gente se deja ll...
4  60004  Pero cuando amanece,y me quedo solo,siento en ...

In [ ]:
test_kaggle.shape

(8599, 2)

# **XG Boost**


## XG Boost 1: BOW


### Algoritmo

#### Paso 1: Preprocesamiento y limpieza de datos

In [ ]:
train['review_es'] = train['review_es'].apply(limpiar_texto)
X, vectorizer = crear_bag_of_words(train['review_es'])
y = train['sentimiento']

Uso LabelEncoder para convertir la columna "sentimiento" a variables numericas para poder usar el modelo de XG Boost

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train['sentimiento'])

#### Paso 2: Division entre train y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Paso 3: Entrenamiento del modelo

In [ ]:
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

#### Paso 4: Evaluo mi modelo

In [ ]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Reporte:\n{report}')

Accuracy: 0.8321
Reporte:
              precision    recall  f1-score   support

           0       0.84      0.81      0.83      4961
           1       0.82      0.85      0.84      5039

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



### Prediccion para el modelo de test dado

In [ ]:
# Preprocesar las nuevas críticas
test_kaggle['review_es_limpio'] = test_kaggle['review_es'].apply(limpiar_texto)

# Convertir las nuevas críticas en Bag of Words utilizando el vectorizador entrenado
X_nuevas_criticas = vectorizer.transform(test_kaggle['review_es_limpio'])

# Predecir el sentimiento con el modelo entrenado
y_nuevas_pred = model.predict(X_nuevas_criticas)

# Convertir las predicciones numéricas a etiquetas originales
test_kaggle['sentimiento'] = label_encoder.inverse_transform(y_nuevas_pred)

# Mostrar el DataFrame con las predicciones
print(test_kaggle[['ID', 'sentimiento']])

         ID sentimiento
0     60000    negativo
1     60001    positivo
2     60002    negativo
3     60003    negativo
4     60004    negativo
...     ...         ...
8594  68594    positivo
8595  68595    negativo
8596  68596    negativo
8597  68597    negativo
8598  68598    negativo

[8599 rows x 2 columns]


In [ ]:
columnas_mostrar = ["ID", "sentimiento"]
XGBoost_BagOfWords = test_kaggle[columnas_mostrar]
XGBoost_BagOfWords.to_csv('XGBoost_BagOfWords.csv', index=False)

## XG Boost 2: TF-IDF

### Algoritmo

#### Paso 1: Preprocesamiento y limpieza de datos

In [ ]:
train['review_es'] = train['review_es'].apply(limpiar_texto)

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(train['review_es']).toarray()
y = train['sentimiento']

#### Paso 2: Division entre train y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Paso 3: Entrenamiento del modelo

In [ ]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

xgb_model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', n_estimators=100)

xgb_model.fit(X_train, y_train_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

#### Paso 4: Evaluo mi modelo

In [ ]:
predictions = xgb_model.predict(X_test)

# Calcular la precisión
accuracy = accuracy_score(y_test_encoded, predictions)
print(f'Accuracy: {accuracy}')

class_report = classification_report(y_test_encoded, predictions, target_names=['Negativo', 'Positivo'])
print('Reporte de Clasificación:')
print(class_report)

Accuracy: 0.8336
Reporte de Clasificación:
              precision    recall  f1-score   support

    Negativo       0.84      0.82      0.83      4961
    Positivo       0.82      0.85      0.84      5039

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



### Prediccion para el modelo de test dado

In [ ]:
test_kaggle['review_es'] = test_kaggle['review_es'].apply(limpiar_texto)
X_nuevo = vectorizer.transform(test_kaggle['review_es']).toarray()

predictions_nuevo = xgb_model.predict(X_nuevo)

test_kaggle['sentimiento'] = label_encoder.inverse_transform(predictions_nuevo)

In [ ]:
columnas_mostrar = ["ID", "sentimiento"]
XGBoost_TF_IDF = test_kaggle[columnas_mostrar]
XGBoost_TF_IDF.to_csv("XGBoost_TF_IDF.csv", index=False)

## XG Boost 3: lemantizacion

### Algoritmo

#### Paso 1: Preprocesamiento de datos

In [ ]:
train['review_es'] = train['review_es'].apply(limpiar_texto_leman)

#### Paso 2: Vectorizacion TF-IDF

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(train['review_es'])

#### Paso 3: Division entre train y test

In [ ]:
le = LabelEncoder()
y = le.fit_transform(train['sentimiento'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Paso 4: Entreno mi modelo

In [ ]:
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

#### Paso 5: Evaluo mi modelo

In [ ]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=le.classes_)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{report}')

Accuracy: 0.8364
Classification Report:
              precision    recall  f1-score   support

    negativo       0.85      0.82      0.83      4961
    positivo       0.83      0.85      0.84      5039

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000



#### Paso 7: Guardo mi modelo

In [ ]:
import joblib

# Guardar el modelo entrenado
joblib.dump(model, 'xgboost_model.pkl')
# Guardar el vectorizador TF-IDF
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
# Guardar el codificador de etiquetas
joblib.dump(le, 'label_encoder.pkl')

['label_encoder.pkl']

### Prediccion para el modelo de test dado

cargo mi modelo ya entrenado

In [ ]:
model = joblib.load('xgboost_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')
le = joblib.load('label_encoder.pkl')

In [ ]:
test_kaggle['review_es'] = test_kaggle['review_es'].apply(limpiar_texto_leman)

X_test_new = vectorizer.transform(test_kaggle['review_es'])

y_pred_new = model.predict(X_test_new)

y_pred_new_labels = le.inverse_transform(y_pred_new)

test_kaggle['sentimiento'] = y_pred_new_labels

In [ ]:
columnas_mostrar = ["ID", "sentimiento"]
XGBoost_leman = test_kaggle[columnas_mostrar]

XGBoost_leman.to_csv('XGBoost_leman.csv', index=False)

## XG Boost 4 : Lemantizacion + BOW

### Algoritmo

#### Paso 1: Preprocesamiento

In [ ]:
train['review_es'] = train['review_es'].apply(limpiar_texto_leman)

#### Paso 2: Vectorizacion BOW

In [ ]:
vectorizer_bow = CountVectorizer(max_features=5000)
X_bow = vectorizer_bow.fit_transform(train['review_es'])

#### Paso 3: Division

In [ ]:
le = LabelEncoder()
y = le.fit_transform(train['sentimiento'])

X_train_bow, X_test_bow, y_train, y_test = train_test_split(X_bow, y, test_size=0.2, random_state=42)

#### Paso 4: Entremo mi modelo

In [ ]:
model_bow = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model_bow.fit(X_train_bow, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

#### Paso 5: Evaluo mi modelo

In [ ]:
y_pred_bow = model_bow.predict(X_test_bow)
accuracy_bow = accuracy_score(y_test, y_pred_bow)
report_bow = classification_report(y_test, y_pred_bow, target_names=le.classes_)

print(f'Accuracy with Bag of Words: {accuracy_bow}')
print(f'Classification Report with Bag of Words:\n{report_bow}')

Accuracy with Bag of Words: 0.8306
Classification Report with Bag of Words:
              precision    recall  f1-score   support

    negativo       0.84      0.81      0.83      4961
    positivo       0.82      0.85      0.83      5039

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



#### Paso 6: Guardo mi modelo

In [ ]:
joblib.dump(model_bow, 'xgboost_model_bow.pkl')
joblib.dump(vectorizer_bow, 'bow_vectorizer.pkl')
joblib.dump(le, 'label_encoder.pkl')

['label_encoder.pkl']

### Prediccion para el modelo de test dado

In [ ]:
model_bow = joblib.load('xgboost_model_bow.pkl')
vectorizer_bow = joblib.load('bow_vectorizer.pkl')
le = joblib.load('label_encoder.pkl')

test_kaggle['review_es'] = test_kaggle['review_es'].apply(limpiar_texto_leman)

X_test_new_bow = vectorizer_bow.transform(test_kaggle['review_es'])

y_pred_new_bow = model_bow.predict(X_test_new_bow)

y_pred_new_bow_labels = le.inverse_transform(y_pred_new_bow)

test_kaggle['sentimiento'] = y_pred_new_bow_labels

XGBoost_leman_bow = test_kaggle[columnas_mostrar]
XGBoost_leman_bow.to_csv('XGBoost_leman_bow.csv', index=False)

## XG Boost 5: Random Search + CV

### Algoritmo

#### Paso 1: Preprocesamiento

In [ ]:
train['review_es'] = train['review_es'].apply(limpiar_texto_leman)

#### Paso 2: Vectorizar

In [ ]:
vectorizer_bow = CountVectorizer(max_features=5000)
X_bow = vectorizer_bow.fit_transform(train['review_es'])

#### Paso 3: Division

In [ ]:
le = LabelEncoder()
y = le.fit_transform(train['sentimiento'])

X_train_bow, X_test_bow, y_train, y_test = train_test_split(X_bow, y, test_size=0.2, random_state=42)

#### Paso 4: Creo mi modelo

In [ ]:
param_grid = {
    'n_estimators': [100, 150, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
grid_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_grid, n_iter=10, cv=3, random_state=42, n_jobs=-1, verbose=2)
grid_search.fit(X_train_bow, y_train)
print(f"Best parameters: {grid_search.best_params_}")

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best parameters: {'subsample': 0.7, 'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.1, 'colsample_bytree': 0.8}


#### Paso 5: Entreno mi modelo

In [ ]:
best_model = grid_search.best_estimator_

#### Paso 6: Evaluo mi modelo

In [ ]:
y_pred_best = best_model.predict(X_test_bow)
accuracy_best = accuracy_score(y_test, y_pred_best)
report_best = classification_report(y_test, y_pred_best, target_names=le.classes_)

print(f'Accuracy with best parameters: {accuracy_best}')
print(f'Classification Report with best parameters:\n{report_best}')

Accuracy with best parameters: 0.8289
Classification Report with best parameters:
              precision    recall  f1-score   support

    negativo       0.85      0.80      0.82      4961
    positivo       0.81      0.86      0.83      5039

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



#### Paso 7: Evaluo mi modelo con cross validation

In [ ]:
cv_scores = cross_val_score(best_model, X_bow, y, cv=5, scoring='accuracy')
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean cross-validation score: {cv_scores.mean()}")

Cross-validation scores: [0.8177 0.825  0.8263 0.8284 0.8249]
Mean cross-validation score: 0.8244600000000002


#### Paso 8: Guardo mi modelo

In [ ]:
# Guardar el mejor modelo
joblib.dump(best_model, 'xgboost_best_model_bow.pkl')
# Guardar el vectorizador Bag of Words
joblib.dump(vectorizer_bow, 'bow_vectorizer.pkl')
# Guardar el codificador de etiquetas
joblib.dump(le, 'label_encoder.pkl')

['label_encoder.pkl']

### Prediccion para el modelo de test dado

In [ ]:
model_bow = joblib.load('xgboost_best_model_bow.pkl')
vectorizer_bow = joblib.load('bow_vectorizer.pkl')
le = joblib.load('label_encoder.pkl')

test_kaggle['review_es'] = test_kaggle['review_es'].apply(limpiar_texto_leman)

X_test_new_bow = vectorizer_bow.transform(test_kaggle['review_es'])

y_pred_new_bow = model_bow.predict(X_test_new_bow)
y_pred_new_bow_labels = le.inverse_transform(y_pred_new_bow)
test_kaggle['sentimiento'] = y_pred_new_bow_labels

XGBoost_leman_bow_gridserch_cv = test_kaggle[columnas_mostrar]
XGBoost_leman_bow_gridserch_cv.to_csv('XGBoost_leman_bow_gridserch_cv.csv', index=False)

## XG Boost 6: Reduccion de la dimensionalidad (PCA)

### Algoritmo

#### Paso 1: Preprocesamiento

In [ ]:
train['review_es'] = train['review_es'].apply(limpiar_texto_leman)

#### Paso 2: Vectorizar (bow)

In [ ]:
vectorizer_bow = CountVectorizer(max_features=5000)
X_bow = vectorizer_bow.fit_transform(train['review_es'])

#### Paso 3: Aplico PCA

In [ ]:
pca = PCA(n_components=100)
X_bow_pca = pca.fit_transform(X_bow.toarray())

#### Paso 4: Division

In [ ]:
le = LabelEncoder()
y = le.fit_transform(train['sentimiento'])

X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_bow_pca, y, test_size=0.2, random_state=42)

#### Paso 5: Entreno mi modelo

In [ ]:
model_pca = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model_pca.fit(X_train_pca, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

#### Paso 6: Evaluo mi modelo

In [ ]:
y_pred_pca = model_pca.predict(X_test_pca)
accuracy_pca = accuracy_score(y_test, y_pred_pca)
report_pca = classification_report(y_test, y_pred_pca, target_names=le.classes_)

print(f'Accuracy with PCA: {accuracy_pca}')
print(f'Classification Report with PCA:\n{report_pca}')

Accuracy with PCA: 0.7869
Classification Report with PCA:
              precision    recall  f1-score   support

    negativo       0.79      0.77      0.78      4961
    positivo       0.78      0.80      0.79      5039

    accuracy                           0.79     10000
   macro avg       0.79      0.79      0.79     10000
weighted avg       0.79      0.79      0.79     10000



#### Paso 7: Guardo mi modelo

In [ ]:
# Guardar el modelo entrenado con PCA
joblib.dump(model_pca, 'xgboost_model_pca.pkl')
# Guardar el vectorizador Bag of Words
joblib.dump(vectorizer_bow, 'bow_vectorizer.pkl')
# Guardar el codificador de etiquetas
joblib.dump(le, 'label_encoder.pkl')
# Guardar el objeto PCA
joblib.dump(pca, 'pca_transformer.pkl')

['pca_transformer.pkl']

### Prediccion para el modelo de test dado

In [ ]:
model_pca = joblib.load('xgboost_model_pca.pkl')
vectorizer_bow = joblib.load('bow_vectorizer.pkl')
le = joblib.load('label_encoder.pkl')
pca = joblib.load('pca_transformer.pkl')

test_kaggle['review_es'] = test_kaggle['review_es'].apply(limpiar_texto_leman)

X_test_bow = vectorizer_bow.transform(test_kaggle['review_es'])

# Reducción de dimensionalidad con PCA
X_test_pca = pca.transform(X_test_bow.toarray())

# Realizar predicciones con el modelo entrenado
y_pred_test = model_pca.predict(X_test_pca)

y_pred_test_labels = le.inverse_transform(y_pred_test)

test_kaggle['sentimiento'] = y_pred_test_labels

XGBoost_PCA = test_kaggle[columnas_mostrar]
XGBoost_PCA.to_csv('XGBoost_PCA.csv', index=False)

##XG Boost 7

### Algoritmo

#### Paso 1: Preprocesamiento de datos

In [ ]:
train['review_clean'] = train['review_es'].apply(clean_text).apply(remove_stopwords).apply(stem_text)

#### Paso 2: Bag of Words

In [ ]:
vectorizer = CountVectorizer(max_features=5000)
X = vectorizer.fit_transform(train['review_clean']).toarray()
y = train['sentimiento']

#### Paso 3: Mapeo de variables a numeros

In [ ]:
label_map = {'negativo': 0, 'positivo': 1}
y_map = [label_map[label] for label in y]

#### Paso 4: Dividir el conjunto en train y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_map, test_size=0.2, random_state=42)

#### Paso 5: Entreno mi modelo

In [ ]:
modelo_XGBoost = xgb.XGBClassifier()
modelo_XGBoost.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
#realizamos nuestras predicciones
y_pred = modelo_XGBoost.predict(X_test)
print(y_pred)

[0 1 0 ... 1 0 1]


#### Paso 6: Evaluo mi modelo

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8377
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.82      0.83      4961
           1       0.83      0.85      0.84      5039

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000



### Prediccion para el modelo de test dado

In [ ]:
# Preprocesar los nuevos datos (limpiar, eliminar stopwords, aplicar stemming, etc.)
test_kaggle['review_clean'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords).apply(stem_text)

# Extraer las características (X) de los nuevos datos
X_nuevos = test_kaggle['review_clean']

In [ ]:
#vectorizamos para poder utilizar nuestro conjunto de palabras en nuestras predicciones.
vectorizer = CountVectorizer(max_features=5000)
X_nuevos = vectorizer.fit_transform(test_kaggle['review_clean']).toarray()

In [ ]:
# Hacer predicciones en los nuevos datos utilizando el modelo entrenado
predicciones_nuevos = modelo_XGBoost.predict(X_nuevos)

In [ ]:
print(predicciones_nuevos)

[1 1 0 ... 0 0 0]


In [ ]:
#mapeamos las predicciones a nuestras variables categoricas nuevamente
map_categoricas = {0: 'negativo', 1: 'positivo'}
y_categorica = [map_categoricas[label] for label in predicciones_nuevos]
print(y_categorica)

['positivo', 'positivo', 'negativo', 'positivo', 'negativo', 'positivo', 'positivo', 'negativo', 'positivo', 'negativo', 'negativo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'negativo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'negativo', 'negativo', 'negativo', 'positivo', 'positivo', 'negativo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'negativo', 'negativo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'negativo', 'negativo', 'negativo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'po

In [ ]:
# Agregar las predicciones al DataFrame de los nuevos datos
test_kaggle['sentimiento'] = y_categorica

#ahora creamos nuestro resultado final
columnas_mostrar = ['ID', 'sentimiento']
resultado_final_XGBoost = test_kaggle[columnas_mostrar]

print(resultado_final_XGBoost)

         ID sentimiento
0     60000    positivo
1     60001    positivo
2     60002    negativo
3     60003    positivo
4     60004    negativo
...     ...         ...
8594  68594    negativo
8595  68595    positivo
8596  68596    negativo
8597  68597    negativo
8598  68598    negativo

[8599 rows x 2 columns]


In [ ]:
resultado_final_XGBoost.to_csv('XGBoost_basico_prueba_2.csv', index=False)

## XG Boost 8: Cambiando hiperparametros

### Algoritmo

#### Paso 1: Preprocesamiento de datos

In [ ]:
train['review_clean'] = train['review_es'].apply(clean_text).apply(remove_stopwords).apply(stem_text)

#### Paso 2: Bag of Words

In [ ]:
vectorizer = CountVectorizer(max_features=5000)
X = vectorizer.fit_transform(train['review_clean']).toarray()
y = train['sentimiento']

#### Paso 3: Mapeo de variables a numeros

In [ ]:
label_map = {'negativo': 0, 'positivo': 1}
y_map = [label_map[label] for label in y]

#### Paso 4: Dividir el conjunto en train y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_map, test_size=0.2, random_state=42)

#### Paso 5: Creo mi modelo

In [ ]:
params = {
    'learning_rate': 0.1,
    'n_estimators': 50,
    'reg_alpha': 0.1,
    'reg_lambda': 0.01,
    'random_state': 1
}

#### Paso 6: Entreno mi modelo

In [ ]:
# Entrenamos el modelo una vez obtenidos los mejores parametros.
modelo_XGBoost = xgb.XGBClassifier(**params)
modelo_XGBoost.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=50, n_jobs=None,
              num_parallel_tree=None, random_state=1, ...)

In [ ]:
#realizamos nuestras predicciones
y_pred = modelo_XGBoost.predict(X_test)
print(y_pred)

[0 1 0 ... 1 1 1]


#### Paso 7: Evaluo mi modelo

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.7837
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.73      0.77      4961
           1       0.76      0.83      0.80      5039

    accuracy                           0.78     10000
   macro avg       0.79      0.78      0.78     10000
weighted avg       0.79      0.78      0.78     10000



### Prediccion para el modelo de test dado

In [ ]:
# Preprocesar los nuevos datos (limpiar, eliminar stopwords, aplicar stemming, etc.)
test_kaggle['review_clean'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords).apply(stem_text)

# Extraer las características (X) de los nuevos datos
X_nuevos = test_kaggle['review_clean']

In [ ]:
#vectorizamos para poder utilizar nuestro conjunto de palabras en nuestras predicciones.
vectorizer = CountVectorizer(max_features=5000)
X_nuevos = vectorizer.fit_transform(test_kaggle['review_clean']).toarray()

In [ ]:
# Hacer predicciones en los nuevos datos utilizando el modelo entrenado
predicciones_nuevos = modelo_XGBoost.predict(X_nuevos)

In [ ]:
print(predicciones_nuevos)

[1 1 1 ... 1 1 1]


In [ ]:
#mapeamos las predicciones a nuestras variables categoricas nuevamente
map_categoricas = {0: 'negativo', 1: 'positivo'}
y_categorica = [map_categoricas[label] for label in predicciones_nuevos]
print(y_categorica)

['positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'po

In [ ]:
# Agregar las predicciones al DataFrame de los nuevos datos
test_kaggle['sentimiento'] = y_categorica

#ahora creamos nuestro resultado final
columnas_mostrar = ['ID', 'sentimiento']
resultado_final_XGBoost = test_kaggle[columnas_mostrar]

print(resultado_final_XGBoost)

         ID sentimiento
0     60000    positivo
1     60001    positivo
2     60002    positivo
3     60003    positivo
4     60004    positivo
...     ...         ...
8594  68594    positivo
8595  68595    positivo
8596  68596    positivo
8597  68597    positivo
8598  68598    positivo

[8599 rows x 2 columns]


In [ ]:
resultado_final_XGBoost.to_csv('XGBoost_basico_probandoHiperparametros_1.csv', index=False)

##XG Boost 9: Cambiando hiperparametros

### Algoritmo

#### Paso 1: Preprocesamiento de datos

In [ ]:
train['review_clean'] = train['review_es'].apply(clean_text).apply(remove_stopwords).apply(stem_text)

#### Paso 2: Bag of Words

In [ ]:
vectorizer = CountVectorizer(max_features=5000)
X = vectorizer.fit_transform(train['review_clean']).toarray()
y = train['sentimiento']

#### Paso 3: Mapeo de variables a numeros

In [ ]:
label_map = {'negativo': 0, 'positivo': 1}
y_map = [label_map[label] for label in y]

#### Paso 4: Dividir el conjunto en train y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_map, test_size=0.2, random_state=42)

#### Paso 5: Creo mi modelo

In [ ]:
params = {
    'learning_rate': 0.2,
    'n_estimators': 100,
    'reg_alpha': 0.1,
    'reg_lambda': 0.01,
    'random_state': 1
}

#### Paso 6: Entreno mi modelo

In [ ]:
modelo_XGBoost = xgb.XGBClassifier(**params)
modelo_XGBoost.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=1, ...)

In [ ]:
y_pred = modelo_XGBoost.predict(X_test)
print(y_pred)

[0 1 0 ... 1 0 1]


#### Paso 7: Evaluo mi modelo

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8277
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.80      0.82      4961
           1       0.82      0.85      0.83      5039

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



### Prediccion para el modelo de test dado

In [ ]:
# Preprocesar los nuevos datos (limpiar, eliminar stopwords, aplicar stemming, etc.)
test_kaggle['review_clean'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords).apply(stem_text)

# Extraer las características (X) de los nuevos datos
X_nuevos = test_kaggle['review_clean']

In [ ]:
#vectorizamos para poder utilizar nuestro conjunto de palabras en nuestras predicciones.
vectorizer = CountVectorizer(max_features=5000)
X_nuevos = vectorizer.fit_transform(test_kaggle['review_clean']).toarray()

In [ ]:
# Hacer predicciones en los nuevos datos utilizando el modelo entrenado
predicciones_nuevos = modelo_XGBoost.predict(X_nuevos)

In [ ]:
print(predicciones_nuevos)

[1 1 0 ... 0 1 0]


In [ ]:
#mapeamos las predicciones a nuestras variables categoricas nuevamente
map_categoricas = {0: 'negativo', 1: 'positivo'}
y_categorica = [map_categoricas[label] for label in predicciones_nuevos]
print(y_categorica)

['positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'negativo', 'negativo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'negativo', 'negativo', 'negativo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'negativo', 'negativo', 'positivo', 'positivo', 'negativo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'negativo', 'negativo', 'negativo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'negativo', 'negativo', 'negativo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'po

In [ ]:
# Agregar las predicciones al DataFrame de los nuevos datos
test_kaggle['sentimiento'] = y_categorica

#ahora creamos nuestro resultado final
columnas_mostrar = ['ID', 'sentimiento']
resultado_final_XGBoost = test_kaggle[columnas_mostrar]

print(resultado_final_XGBoost)

         ID sentimiento
0     60000    positivo
1     60001    positivo
2     60002    negativo
3     60003    positivo
4     60004    positivo
...     ...         ...
8594  68594    negativo
8595  68595    positivo
8596  68596    negativo
8597  68597    positivo
8598  68598    negativo

[8599 rows x 2 columns]


In [ ]:
resultado_final_XGBoost.to_csv('XGBoost_basico_probandoHiperparametros_2.csv', index=False)

Conclusion: Al subirlo a kaggle observamos que la performance de este ultimo modelo es peor al anterior, por lo que nos lleva a sospechar de un caso de over-fitting, aumentamos los parametros de regularizacion y vemos una vez mas.

##XG Boost 10: Cambiando hiperparametros

### Algoritmo

#### Paso 1: Preprocesamiento de datos

In [ ]:
train['review_clean'] = train['review_es'].apply(clean_text).apply(remove_stopwords).apply(stem_text)

#### Paso 2: Bag of Words

In [ ]:
vectorizer = CountVectorizer(max_features=5000)
X = vectorizer.fit_transform(train['review_clean']).toarray()
y = train['sentimiento']

#### Paso 3: Mapeo de variables a numeros

In [ ]:
label_map = {'negativo': 0, 'positivo': 1}
y_map = [label_map[label] for label in y]

#### Paso 4: Dividir el conjunto en train y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_map, test_size=0.2, random_state=42)

#### Paso 5: Creo mi modelo

In [ ]:
params = {
    'learning_rate': 0.2,
    'n_estimators': 150,
    'reg_alpha': 0.2,
    'reg_lambda': 0.1,
    'random_state': 1
}

#### Paso 6: Entreno mi modelo

In [ ]:
# Entrenamos el modelo una vez obtenidos los mejores parametros.
modelo_XGBoost = xgb.XGBClassifier(**params)
modelo_XGBoost.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=150, n_jobs=None,
              num_parallel_tree=None, random_state=1, ...)

In [ ]:
#realizamos nuestras predicciones
y_pred = modelo_XGBoost.predict(X_test)
print(y_pred)

[1 1 0 ... 1 0 1]


####Paso 7: Evaluo mi modelo

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8398
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.82      0.84      4961
           1       0.83      0.86      0.84      5039

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000



### Prediccion para el modelo de test dado

In [ ]:
test_kaggle['review_clean'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords).apply(stem_text)

# Extraer las características (X) de los nuevos datos
X_nuevos = test_kaggle['review_clean']

In [ ]:
#vectorizamos para poder utilizar nuestro conjunto de palabras en nuestras predicciones.
vectorizer = CountVectorizer(max_features=5000)
X_nuevos = vectorizer.fit_transform(test_kaggle['review_clean']).toarray()

In [ ]:
# Hacer predicciones en los nuevos datos utilizando el modelo entrenado
predicciones_nuevos = modelo_XGBoost.predict(X_nuevos)

In [ ]:
print(predicciones_nuevos)

[1 1 0 ... 0 0 0]


In [ ]:
#mapeamos las predicciones a nuestras variables categoricas nuevamente
map_categoricas = {0: 'negativo', 1: 'positivo'}
y_categorica = [map_categoricas[label] for label in predicciones_nuevos]
print(y_categorica)

['positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'negativo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'negativo', 'negativo', 'negativo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'negativo', 'negativo', 'negativo', 'positivo', 'positivo', 'positivo', 'negativo', 'negativo', 'negativo', 'positivo', 'positivo', 'negativo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'negativo', 'negativo', 'negativo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'po

In [ ]:
# Agregar las predicciones al DataFrame de los nuevos datos
test_kaggle['sentimiento'] = y_categorica

#ahora creamos nuestro resultado final
columnas_mostrar = ['ID', 'sentimiento']
resultado_final_XGBoost = test_kaggle[columnas_mostrar]

print(resultado_final_XGBoost)

         ID sentimiento
0     60000    positivo
1     60001    positivo
2     60002    negativo
3     60003    positivo
4     60004    positivo
...     ...         ...
8594  68594    negativo
8595  68595    positivo
8596  68596    negativo
8597  68597    negativo
8598  68598    negativo

[8599 rows x 2 columns]


In [ ]:
resultado_final_XGBoost.to_csv('XGBoost_basico_probandoHiperparametros_3.csv', index=False)

Conclusion: Seguimos probando hiperparametros, el que mejor performance nos arrojo durante el conjunto de test de kaggle fue el primero de los 3, veamos si podemos encontrar porque es eso

## XG Boost 11: Cambiando hiperparametros

### Algoritmo

#### Paso 1: Preprocesamiento de datos

In [ ]:
train['review_clean'] = train['review_es'].apply(clean_text).apply(remove_stopwords).apply(stem_text)

#### Paso 2: Bag of Words

In [ ]:
vectorizer = CountVectorizer(max_features=5000)
X = vectorizer.fit_transform(train['review_clean']).toarray()
y = train['sentimiento']

#### Paso 3: Mapeo de variables a numeros

In [ ]:
label_map = {'negativo': 0, 'positivo': 1}
y_map = [label_map[label] for label in y]

#### Paso 4: Dividir el conjunto en train y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_map, test_size=0.2, random_state=42)

#### Paso 5: Creo mi modelo

In [ ]:
params = {
    'learning_rate': 0.2,
    'n_estimators': 40,
    'reg_alpha': 0.1,
    'reg_lambda': 0.01,
    'random_state': 1,
    'max_depth': 5,
}

#### Paso 6: Entreno mi modelo

In [ ]:
modelo_XGBoost = xgb.XGBClassifier(**params)
modelo_XGBoost.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=40, n_jobs=None,
              num_parallel_tree=None, random_state=1, ...)

In [ ]:
#realizamos nuestras predicciones
y_pred = modelo_XGBoost.predict(X_test)
print(y_pred)

[0 1 0 ... 1 0 1]


####Paso 7: Evaluo mi modelo

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.7928
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.75      0.78      4961
           1       0.77      0.83      0.80      5039

    accuracy                           0.79     10000
   macro avg       0.79      0.79      0.79     10000
weighted avg       0.79      0.79      0.79     10000



### Prediccion para el modelo de test dado

In [ ]:
test_kaggle['review_clean'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords).apply(stem_text)

# Extraer las características (X) de los nuevos datos
X_nuevos = test_kaggle['review_clean']

In [ ]:
#vectorizamos para poder utilizar nuestro conjunto de palabras en nuestras predicciones.
vectorizer = CountVectorizer(max_features=5000)
X_nuevos = vectorizer.fit_transform(test_kaggle['review_clean']).toarray()

In [ ]:
# Hacer predicciones en los nuevos datos utilizando el modelo entrenado
predicciones_nuevos = modelo_XGBoost.predict(X_nuevos)

In [ ]:
print(predicciones_nuevos)

[1 1 1 ... 1 1 1]


In [ ]:
#mapeamos las predicciones a nuestras variables categoricas nuevamente
map_categoricas = {0: 'negativo', 1: 'positivo'}
y_categorica = [map_categoricas[label] for label in predicciones_nuevos]
print(y_categorica)

['positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'negativo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'po

In [ ]:
# Agregar las predicciones al DataFrame de los nuevos datos
test_kaggle['sentimiento'] = y_categorica

#ahora creamos nuestro resultado final
columnas_mostrar = ['ID', 'sentimiento']
resultado_final_XGBoost = test_kaggle[columnas_mostrar]

print(resultado_final_XGBoost)

         ID sentimiento
0     60000    positivo
1     60001    positivo
2     60002    positivo
3     60003    positivo
4     60004    positivo
...     ...         ...
8594  68594    positivo
8595  68595    positivo
8596  68596    positivo
8597  68597    positivo
8598  68598    positivo

[8599 rows x 2 columns]


In [ ]:
resultado_final_XGBoost.to_csv('XGBoost_basico_probandoHiperparametros_4.csv', index=False)

## XG Boost 12: Cambiando hiperparametros

### Algoritmo

#### Paso 1: Preprocesamiento de datos

In [ ]:
train['review_clean'] = train['review_es'].apply(clean_text).apply(remove_stopwords).apply(stem_text)

#### Paso 2: Bag of Words

In [ ]:
vectorizer = CountVectorizer(max_features=5000)
X = vectorizer.fit_transform(train['review_clean']).toarray()
y = train['sentimiento']

#### Paso 3: Mapeo de variables a numeros

In [ ]:
label_map = {'negativo': 0, 'positivo': 1}
y_map = [label_map[label] for label in y]

#### Paso 4: Dividir el conjunto en train y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_map, test_size=0.2, random_state=42)

#### Paso 5: Cambiando hiperparametros

In [ ]:
params = {
    'learning_rate': 1,
    'n_estimators': 200,
    'reg_alpha': 0,
    'reg_lambda': 0,
    'random_state': 1,
    'max_depth': 40
}

#### Paso 6: Entreno mi modelo

In [ ]:
modelo_XGBoost = xgb.XGBClassifier(**params)
modelo_XGBoost.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=40, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=1, ...)

In [ ]:
#realizamos nuestras predicciones
y_pred = modelo_XGBoost.predict(X_test)
print(y_pred)

[0 1 0 ... 1 0 1]


#### Paso 7: Evaluo mi modelo

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8251
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.82      0.82      4961
           1       0.82      0.83      0.83      5039

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



### Prediccion para el modelo de test dado

In [ ]:
test_kaggle['review_clean'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords).apply(stem_text)

# Extraer las características (X) de los nuevos datos
X_nuevos = test_kaggle['review_clean']

In [ ]:
#vectorizamos para poder utilizar nuestro conjunto de palabras en nuestras predicciones.
vectorizer = CountVectorizer(max_features=5000)
X_nuevos = vectorizer.fit_transform(test_kaggle['review_clean']).toarray()

In [ ]:
# Hacer predicciones en los nuevos datos utilizando el modelo entrenado
predicciones_nuevos = modelo_XGBoost.predict(X_nuevos)

In [ ]:
print(predicciones_nuevos)

[0 1 0 ... 0 1 0]


In [ ]:
#mapeamos las predicciones a nuestras variables categoricas nuevamente
map_categoricas = {0: 'negativo', 1: 'positivo'}
y_categorica = [map_categoricas[label] for label in predicciones_nuevos]
print(y_categorica)

['negativo', 'positivo', 'negativo', 'positivo', 'negativo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'negativo', 'negativo', 'negativo', 'negativo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'negativo', 'positivo', 'negativo', 'negativo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'negativo', 'positivo', 'negativo', 'negativo', 'negativo', 'negativo', 'positivo', 'negativo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'negativo', 'positivo', 'negativo', 'negativo', 'negativo', 'negativo', 'negativo', 'negativo', 'positivo', 'negativo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'negativo', 'negativo', 'negativo', 'negativo', 'negativo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'negativo', 'negativo', 'positivo', 'negativo', 'po

In [ ]:
# Agregar las predicciones al DataFrame de los nuevos datos
test_kaggle['sentimiento'] = y_categorica

#ahora creamos nuestro resultado final
columnas_mostrar = ['ID', 'sentimiento']
resultado_final_XGBoost = test_kaggle[columnas_mostrar]

print(resultado_final_XGBoost)

         ID sentimiento
0     60000    negativo
1     60001    positivo
2     60002    negativo
3     60003    positivo
4     60004    negativo
...     ...         ...
8594  68594    positivo
8595  68595    negativo
8596  68596    negativo
8597  68597    positivo
8598  68598    negativo

[8599 rows x 2 columns]


In [ ]:
resultado_final_XGBoost.to_csv('XGBoost_basico_probandoHiperparametros_final.csv', index=False)

Conclusion: viendo los resultados en kaggle, necesitamos los parametros de regularizacion, y no aumentar tanto el maximo de profundidad o la cantidad de estimadores en nuestros modelos de XGBoost. Es decir, tenemos un claro caso de over-fitting, tener cuidado a la hora de

## XG Boost 13: Aumentar BOW, + hiperparametros y trigamas

Para este XGBoost vamos a probar aumentando el tamaño del Bag of Words, ajustando y agregando hiperparámetros de XGBoost y utilizando trigramas en la vectorización.

### Algoritmo

#### Paso 1: Preprocesamiento y limpieza de datos

In [ ]:
train['review_es'] = train['review_es'].apply(limpiar_texto)
X, vectorizer = crear_bag_of_words(train['review_es'])
y = train['sentimiento']

Uso LabelEncoder para convertir la columna "sentimiento" a variables numericas para poder usar el modelo de XG Boost

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train['sentimiento'])

#### Paso 2: Division entre train y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Paso 3: Entrenamiento del modelo

In [ ]:
model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    n_estimators=200,         # Aumentar el número de árboles
    learning_rate=0.05,       # Disminuir la tasa de aprendizaje
    max_depth=10,             # Aumentar la profundidad máxima de los árboles
    colsample_bytree=0.8,     # Submuestrear las columnas por árbol
    subsample=0.8             # Submuestrear las filas por árbol
)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

#### Paso 4: Evaluo mi modelo

In [ ]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Reporte de Clasificacion:\n{report}')

Accuracy: 0.8256
Reporte de Clasificacion:
              precision    recall  f1-score   support

           0       0.84      0.80      0.82      4961
           1       0.81      0.85      0.83      5039

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



### Prediccion para el modelo de test dado

In [ ]:
# Preprocesar las nuevas críticas
test_kaggle['review_es_limpio'] = test_kaggle['review_es'].apply(limpiar_texto)

# Convertir las nuevas críticas en Bag of Words utilizando el vectorizador entrenado
X_nuevas_criticas = vectorizer.transform(test_kaggle['review_es_limpio'])

# Predecir el sentimiento con el modelo entrenado
y_nuevas_pred = model.predict(X_nuevas_criticas)

# Convertir las predicciones numéricas a etiquetas originales
test_kaggle['sentimiento'] = label_encoder.inverse_transform(y_nuevas_pred)

# Mostrar el DataFrame con las predicciones
print(test_kaggle[['ID', 'sentimiento']])

         ID sentimiento
0     60000    negativo
1     60001    positivo
2     60002    negativo
3     60003    negativo
4     60004    negativo
...     ...         ...
8594  68594    positivo
8595  68595    negativo
8596  68596    negativo
8597  68597    negativo
8598  68598    positivo

[8599 rows x 2 columns]


In [ ]:
columnas_mostrar = ['ID', 'sentimiento']
resultado = test_kaggle[columnas_mostrar]
resultado.to_csv('XG_Boost_trigams.csv', index=False)

## XG Boost 14: + hiperparametros

Para este XGBoost vamos a mantener las modificaciones previas y cambiar y agregar aún más hiperparametros al XG Boost para intentar obtener una mejor f1-score.

### Algoritmo

#### Paso 1: Preprocesamiento y limpieza de datos

In [ ]:
train['review_es'] = train['review_es'].apply(limpiar_texto)
X, vectorizer = crear_bag_of_words(train['review_es'])
y = train['sentimiento']

Uso LabelEncoder para convertir la columna "sentimiento" a variables numericas para poder usar el modelo de XG Boost

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train['sentimiento'])

#### Paso 2: Division entre train y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Paso 3: Entrenamiento del modelo

In [ ]:
model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    n_estimators=500,         # Aumentar el número de árboles
    learning_rate=0.05,       # Reducir la tasa de aprendizaje
    max_depth=10,             # Aumentar la profundidad máxima de los árboles
    colsample_bytree=0.8,     # Ajustar el submuestreo de columnas por árbol
    subsample=0.8,            # Ajustar el submuestreo de filas por árbol
    min_child_weight=1,       # Ajustar el peso mínimo de la muestra por nodo
    gamma=0.2,                # Ajustar el parámetro de regularización gamma
    reg_alpha=0.1,            # Ajustar la regularización L1
    reg_lambda=1.0            # Ajustar la regularización L2
)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=0.2, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

#### Paso 4: Evaluo mi modelo

In [ ]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Reporte de Clasificacion:\n{report}')

Accuracy: 0.8447
Reporte de Clasificacion:
              precision    recall  f1-score   support

           0       0.85      0.84      0.84      4961
           1       0.84      0.85      0.85      5039

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000



### Prediccion para el modelo de test dado

In [ ]:
# Preprocesar las nuevas críticas
test_kaggle['review_es_limpio'] = test_kaggle['review_es'].apply(limpiar_texto)

# Convertir las nuevas críticas en Bag of Words utilizando el vectorizador entrenado
X_nuevas_criticas = vectorizer.transform(test_kaggle['review_es_limpio'])

# Predecir el sentimiento con el modelo entrenado
y_nuevas_pred = model.predict(X_nuevas_criticas)

# Convertir las predicciones numéricas a etiquetas originales
test_kaggle['sentimiento'] = label_encoder.inverse_transform(y_nuevas_pred)

# Mostrar el DataFrame con las predicciones
print(test_kaggle[['ID', 'sentimiento']])

         ID sentimiento
0     60000    negativo
1     60001    positivo
2     60002    positivo
3     60003    negativo
4     60004    negativo
...     ...         ...
8594  68594    positivo
8595  68595    negativo
8596  68596    negativo
8597  68597    negativo
8598  68598    negativo

[8599 rows x 2 columns]


In [ ]:
# Guardar el DataFrame con las predicciones a un nuevo archivo CSV
columnas_mostrar = ["ID", "sentimiento"]
XGBoost_BagOfWords = test_kaggle[columnas_mostrar]
XGBoost_BagOfWords.to_csv('XGBoost_BagOfWords_trigams_v2.csv', index=False)

Vemos que los resultados mejoran notoriamente pero tenemos que tener cuidado a la hora de modificar los hiperparámetros ya que podemos caer en el caso de overfitting.

## XG Boost 15: TF-IDF, + hiperparametros, chi-square

Para este XGBoost vamos a probar otra estrategia: el uso de TF-IDF para vectorizar el texto en lugar de Bag Of Words, pero (a diferencia de antes) utilizando la técnica de selección de características usando Chi-Square para reducir la dimensionalidad y mejorar el rendimiento del modelo.

### Algoritmo

#### Paso 1: Preprocesamiento y limpieza de datos

In [ ]:
train['review_es'] = train['review_es'].apply(limpiar_texto)
X, vectorizer = crear_tfidf(train['review_es'])
y = train['sentimiento']

Uso LabelEncoder para convertir la columna "sentimiento" a variables numericas para poder usar el modelo de XG Boost

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train['sentimiento'])
def crear_tfidf(textos):
    vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))  # Usar TF-IDF con bigramas
    X = vectorizer.fit_transform(textos)
    return X, vectorizer

Selecciono características usando Chi-Square

In [ ]:
selector = SelectKBest(chi2, k=5000)  # Seleccionar las 5000 mejores características
X = selector.fit_transform(X, y)

#### Paso 2: Division entre train y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Paso 3: Entrenamiento del modelo

In [ ]:
model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    n_estimators=300,
    learning_rate=0.1,
    max_depth=7,
    colsample_bytree=0.9,
    subsample=0.9
)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=7,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=300,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

#### Paso 4: Evaluo mi modelo

In [ ]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Reporte de Clasificacion:\n{report}')

Accuracy: 0.8422
Reporte de Clasificacion:
              precision    recall  f1-score   support

           0       0.85      0.83      0.84      4961
           1       0.83      0.86      0.85      5039

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000



### Prediccion para el modelo de test dado

In [ ]:
test_kaggle['review_es_limpio'] = test_kaggle['review_es'].apply(limpiar_texto)

# Convertir las nuevas críticas en TF-IDF utilizando el vectorizador entrenado
X_nuevas_criticas = vectorizer.transform(test_kaggle['review_es_limpio'])
X_nuevas_criticas = selector.transform(X_nuevas_criticas)  # Aplicar selección de características

# Predecir el sentimiento con el modelo entrenado
y_nuevas_pred = model.predict(X_nuevas_criticas)

# Convertir las predicciones numéricas a etiquetas originales
test_kaggle['sentimiento'] = label_encoder.inverse_transform(y_nuevas_pred)

# Mostrar el DataFrame con las predicciones
print(test_kaggle[['ID', 'sentimiento']])

         ID sentimiento
0     60000    negativo
1     60001    positivo
2     60002    positivo
3     60003    negativo
4     60004    negativo
...     ...         ...
8594  68594    positivo
8595  68595    negativo
8596  68596    negativo
8597  68597    negativo
8598  68598    negativo

[8599 rows x 2 columns]


In [ ]:
# Guardar el DataFrame con las predicciones a un nuevo archivo CSV
columnas_mostrar = ["ID", "sentimiento"]
XGBoost_TFIDF = test_kaggle[columnas_mostrar]
XGBoost_TFIDF.to_csv('XGBoost_TFIDF_chi2.csv', index=False)

## XG Boost 16: modelo 15 + modelo 14

Para este XGBoost vamos a ir con la misma estrategia que el modelo 15 solo que, para crear el XG Boost, utilizamos los parametros del modelo 14.

### Algoritmo

#### Paso 1: Preprocesamiento y limpieza de datos

In [ ]:
train['review_es'] = train['review_es'].apply(limpiar_texto)
X, vectorizer = crear_tfidf(train['review_es'])
y = train['sentimiento']

Uso LabelEncoder para convertir la columna "sentimiento" a variables numericas para poder usar el modelo de XG Boost

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train['sentimiento'])

Selecciono características usando Chi-Square

In [ ]:
selector = SelectKBest(chi2, k=5000)  # Seleccionar las 5000 mejores características
X = selector.fit_transform(X, y)

#### Paso 2: Division entre train y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Paso 3: Entrenamiento del modelo

In [ ]:
model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    n_estimators=500,         # Aumentar el número de árboles
    learning_rate=0.05,       # Reducir la tasa de aprendizaje
    max_depth=10,             # Aumentar la profundidad máxima de los árboles
    colsample_bytree=0.8,     # Ajustar el submuestreo de columnas por árbol
    subsample=0.8,            # Ajustar el submuestreo de filas por árbol
    min_child_weight=1,       # Ajustar el peso mínimo de la muestra por nodo
    gamma=0.2,                # Ajustar el parámetro de regularización gamma
    reg_alpha=0.1,            # Ajustar la regularización L1
    reg_lambda=1.0            # Ajustar la regularización L2
)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=0.2, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

#### Paso 4: Evaluo mi modelo

In [ ]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Reporte de Clasificacion:\n{report}')

Accuracy: 0.8515
Reporte de Clasificacion:
              precision    recall  f1-score   support

           0       0.86      0.84      0.85      4961
           1       0.85      0.86      0.85      5039

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



### Prediccion para el modelo de test dado

In [ ]:
# Preprocesar las nuevas críticas
test_kaggle['review_es_limpio'] = test_kaggle['review_es'].apply(limpiar_texto)

# Convertir las nuevas críticas en TF-IDF utilizando el vectorizador entrenado
X_nuevas_criticas = vectorizer.transform(test_kaggle['review_es_limpio'])
X_nuevas_criticas = selector.transform(X_nuevas_criticas)  # Aplicar selección de características

# Predecir el sentimiento con el modelo entrenado
y_nuevas_pred = model.predict(X_nuevas_criticas)

# Convertir las predicciones numéricas a etiquetas originales
test_kaggle['sentimiento'] = label_encoder.inverse_transform(y_nuevas_pred)

# Mostrar el DataFrame con las predicciones
print(test_kaggle[['ID', 'sentimiento']])

         ID sentimiento
0     60000    negativo
1     60001    positivo
2     60002    negativo
3     60003    negativo
4     60004    negativo
...     ...         ...
8594  68594    positivo
8595  68595    negativo
8596  68596    positivo
8597  68597    negativo
8598  68598    negativo

[8599 rows x 2 columns]


In [ ]:
# Guardar el DataFrame con las predicciones a un nuevo archivo CSV
columnas_mostrar = ["ID", "sentimiento"]
XGBoost_TFIDF = test_kaggle[columnas_mostrar]
XGBoost_TFIDF.to_csv('XGBoost_TFIDF_chi2_v2.csv', index=False)

## XG Boost 17: modelo 14 sin stemming

Para este XGBoost vamos a mantener las modificaciones previas y cambiar y agregar aún más hiperparametros al XG Boost para intentar obtener una mejor f1-score.

### Algoritmo

#### Paso 1: Preprocesamiento y limpieza de datos

In [ ]:
train['review_es'] = train['review_es'].apply(clean_text).apply(remove_stopwords)
X, vectorizer = crear_bag_of_words(train['review_es'])
y = train['sentimiento']

Uso LabelEncoder para convertir la columna "sentimiento" a variables numericas para poder usar el modelo de XG Boost

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train['sentimiento'])

#### Paso 2: Division entre train y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Paso 3: Entrenamiento del modelo

In [ ]:
model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    n_estimators=500,         # Aumentar el número de árboles
    learning_rate=0.05,       # Reducir la tasa de aprendizaje
    max_depth=10,             # Aumentar la profundidad máxima de los árboles
    colsample_bytree=0.8,     # Ajustar el submuestreo de columnas por árbol
    subsample=0.8,            # Ajustar el submuestreo de filas por árbol
    min_child_weight=1,       # Ajustar el peso mínimo de la muestra por nodo
    gamma=0.2,                # Ajustar el parámetro de regularización gamma
    reg_alpha=0.1,            # Ajustar la regularización L1
    reg_lambda=1.0            # Ajustar la regularización L2
)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=0.2, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

#### Paso 4: Evaluo mi modelo

In [ ]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Reporte de Clasificacion:\n{report}')

Accuracy: 0.8463
Reporte de Clasificacion:
              precision    recall  f1-score   support

           0       0.85      0.84      0.84      4961
           1       0.84      0.86      0.85      5039

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



### Prediccion para el modelo de test dado

In [ ]:
# Preprocesar las nuevas críticas
test_kaggle['review_es_limpio'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords)

# Convertir las nuevas críticas en Bag of Words utilizando el vectorizador entrenado
X_nuevas_criticas = vectorizer.transform(test_kaggle['review_es_limpio'])

# Predecir el sentimiento con el modelo entrenado
y_nuevas_pred = model.predict(X_nuevas_criticas)

# Convertir las predicciones numéricas a etiquetas originales
test_kaggle['sentimiento'] = label_encoder.inverse_transform(y_nuevas_pred)

# Mostrar el DataFrame con las predicciones
print(test_kaggle[['ID', 'sentimiento']])

         ID sentimiento
0     60000    negativo
1     60001    positivo
2     60002    negativo
3     60003    negativo
4     60004    negativo
...     ...         ...
8594  68594    positivo
8595  68595    negativo
8596  68596    negativo
8597  68597    negativo
8598  68598    negativo

[8599 rows x 2 columns]


In [ ]:
# Guardar el DataFrame con las predicciones a un nuevo archivo CSV
columnas_mostrar = ["ID", "sentimiento"]
XGBoost_BagOfWords = test_kaggle[columnas_mostrar]
XGBoost_BagOfWords.to_csv('XGBoost_BagOfWords_trigams_v3.csv', index=False)

## XG Boost 18: modelo 2 pero sin stemming

### Algoritmo

#### Paso 1: Preprocesamiento y limpieza de datos

In [ ]:
train['review_es'] = train['review_es'].apply(clean_text).apply(remove_stopwords)

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(train['review_es']).toarray()
y = train['sentimiento']

#### Paso 2: Division entre train y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Paso 3: Entrenamiento del modelo

In [ ]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

xgb_model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', n_estimators=100)

xgb_model.fit(X_train, y_train_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

#### Paso 4: Evaluo mi modelo

In [ ]:
predictions = xgb_model.predict(X_test)

# Calcular la precisión
accuracy = accuracy_score(y_test_encoded, predictions)
print(f'Accuracy: {accuracy}')

class_report = classification_report(y_test_encoded, predictions, target_names=['Negativo', 'Positivo'])
print('Reporte de Clasificación:')
print(class_report)

Accuracy: 0.8308
Reporte de Clasificación:
              precision    recall  f1-score   support

    Negativo       0.84      0.82      0.83      4961
    Positivo       0.82      0.85      0.83      5039

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



### Prediccion para el modelo de test dado

In [ ]:
test_kaggle['review_es'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords)
X_nuevo = vectorizer.transform(test_kaggle['review_es']).toarray()

predictions_nuevo = xgb_model.predict(X_nuevo)

test_kaggle['sentimiento'] = label_encoder.inverse_transform(predictions_nuevo)

In [ ]:
columnas_mostrar = ["ID", "sentimiento"]
XGBoost_TF_IDF = test_kaggle[columnas_mostrar]
XGBoost_TF_IDF.to_csv("XGBoost_TF_IDF_2.0.csv", index=False)